In [ ]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt


import sklearn.utils as slu



In [ ]:
# resampled train set, test set left unsampled
train_samp = np.loadtxt('./train_test_sampled/train.csv', delimiter=',')
test_samp = np.loadtxt('./train_test_sampled/test.csv', delimiter=',')

print(train_samp.shape)
print(test_samp.shape)

X_train = train_samp[:,:-3]
y_train = train_samp[:,-3]

X_test = test_samp[:,:-3]
y_test = test_samp[:,-3]

## CF Matrix and Scores Functions

In [1]:
from sklearn.metrics import *
import seaborn as sns
def cf_matrix_func(predictions, y_test):
    cm = confusion_matrix(y_test, predictions)
    categories=['N', 'L', 'R', 'A', 'V', 'F', 'f','P']
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    sns.heatmap(cm, annot=True, 
                fmt='.2%', cmap='Blues',xticklabels=categories,yticklabels=categories)
    plt.xlabel('predicted label')
    plt.ylabel('true label')
    
def scores(predictions, y_test):
    accuracy = accuracy_score(y_test, predictions)
    precision_micro = precision_score(y_test, predictions, average='micro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
    precision_macro = precision_score(y_test, predictions, average='macro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
    f1_macro = f1_score(y_test, predictions, average='macro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
    f1_micro = f1_score(y_test, predictions, average='micro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
    print("accuracy: ", accuracy, "precision_macro: ", precision_macro, "precision_micro: ", precision_micro, "f1_macro: ", f1_macro, "f1_micro: ", f1_micro)
    return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

## CNN Classifier

In [2]:
from keras.utils.np_utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

Using TensorFlow backend.


In [ ]:
X_train_3d = X_train.reshape(-1, X_train.shape[1],1)
X_test_3d = X_test.reshape(-1, X_test.shape[1], 1)

print(X_train_3d.shape)
print(X_test_3d.shape)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
print(y_train_cat.shape)
print(y_test_cat.shape)

In [ ]:
verbose, epoch, batch_size = 1, 10, 128

def getModel():
    cnnmodel = Sequential()
    cnnmodel.add(Conv1D(filters=128, kernel_size=2, activation='relu',input_shape=(X_train_3d.shape[1],X_train_3d.shape[2])))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Dropout(rate=0.2))
    cnnmodel.add(Flatten())
    cnnmodel.add(Dense(64, activation='relu'))
    cnnmodel.add(Dense(32, activation='relu'))
    cnnmodel.add(Dense(9, activation='softmax'))
    cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    cnnmodel.summary()
    return cnnmodel

cnnmodel = getModel()

In [ ]:
model = cnnmodel.fit(X_train_3d, y_train_cat, epochs=epoch, validation_split=0.2, verbose=verbose,batch_size=batch_size)

In [ ]:
cnnpredictions = cnnmodel.predict(X_test_3d, verbose=1)

In [ ]:
cnn_predict = np.argmax(cnnpredictions, axis=1)
true_value = np.argmax(y_test_cat,axis=1)

In [ ]:
score = scores(true_value, cnn_predict)

target_names = ['N','L','R','A','V','F','f','/']

print(classification_report(true_value, cnn_predict, target_names=target_names))

cf_matrix_func(cnn_predict,true_value)
